
## 作業
礙於不是所有同學都有 GPU ，這邊的範例使用的是簡化版本的 ResNet，確保所有同學都能夠順利訓練!


最後一天的作業請閱讀這篇非常詳盡的[文章](https://blog.gtwang.org/programming/keras-resnet-50-pre-trained-model-build-dogs-cats-image-classification-system/)，基本上已經涵蓋了所有訓練　CNN 常用的技巧，請使用所有學過的訓練技巧，盡可能地提高 Cifar-10 的 test data 準確率，截圖你最佳的結果並上傳來完成最後一次的作業吧!

另外這些技巧在 Kaggle 上也會被許多人使用，更有人會開發一些新的技巧，例如使把預訓練在 ImageNet 上的模型當成 feature extractor 後，再拿擷取出的特徵重新訓練新的模型，這些技巧再進階的課程我們會在提到，有興趣的同學也可以[參考](https://www.kaggle.com/insaff/img-feature-extraction-with-pretrained-resnet)

In [1]:
import os
import numpy as np
from math import ceil
from keras.datasets import cifar10
from keras.models import Model
from keras.optimizers import Adam
from keras.utils import to_categorical
from keras.layers import Flatten, Dense
from keras.applications.resnet50 import ResNet50
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ReduceLROnPlateau, ModelCheckpoint
batch_size = 100 # batch 的大小，如果出現 OOM error，請降低這個值
num_classes = 10 # 類別的數量，Cifar 10 共有 10 個類別
epochs = 32 # 訓練整個資料集共 30個循環
save_dir = os.path.join(os.getcwd(), 'saved_models')
model_name = 'cifar10_ResNet50_model.h5'
if not os.path.isdir(save_dir):
    os.makedirs(save_dir)
filepath = os.path.join(save_dir, model_name)

Using TensorFlow backend.


In [2]:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')
x_train = x_train.astype('float32')/ 255.
x_test = x_test.astype('float32')/ 255.
x_train_mean = np.mean(x_train, axis=0)
x_train -= x_train_mean
x_test -= x_train_mean
y_train = to_categorical(y_train, num_classes)
y_test = to_categorical(y_test, num_classes)

x_train shape: (50000, 32, 32, 3)
50000 train samples
10000 test samples


In [0]:
datagen = ImageDataGenerator(width_shift_range=0.1, height_shift_range=0.1, zca_epsilon=1e-06, horizontal_flip=True)
datagen.fit(x_train)
checkpoint = ModelCheckpoint(filepath=filepath,monitor='val_acc', save_best_only=True)
reduce_lr = ReduceLROnPlateau()

In [4]:
net = ResNet50(include_top=False, weights='imagenet', input_tensor=None, input_shape=(32,32,3))
x = net.output
x = Flatten()(x)
output_layer = Dense(num_classes, activation='softmax', name='softmax')(x)
model = Model(inputs=net.input, outputs=output_layer)
model.compile(loss='categorical_crossentropy', optimizer=Adam(), metrics=['accuracy'])
model.summary()

/usr/local/lib/python3.6/dist-packages/keras_applications/resnet50.py:265: UserWarning: The output shape of `ResNet50(include_top=False)` has been changed since Keras 2.2.0.
  warnings.warn('The output shape of `ResNet50(include_top=False)` '





Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 32, 32, 3)    0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 38, 38, 3)    0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, 16, 16, 64)   9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_conv1 (BatchNormalization)   (None, 16, 16, 64)   256         conv1[0][0]                      
_________________________________________________________________________________________

In [5]:
history = model.fit_generator(datagen.flow(x_train, y_train, batch_size=batch_size), validation_data = (x_test, y_test), epochs = epochs, 
                              steps_per_epoch=ceil(x_train.shape[0]/batch_size),callbacks=[reduce_lr,checkpoint])
score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Epoch 1/32
500/500 [==============================] - 77s 153ms/step - loss: 1.4333 - acc: 0.5624 - val_loss: 1.1441 - val_acc: 0.6614
Epoch 2/32
500/500 [==============================] - 63s 125ms/step - loss: 1.6761 - acc: 0.4504 - val_loss: 1.7835 - val_acc: 0.4442
Epoch 3/32
500/500 [==============================] - 63s 126ms/step - loss: 1.3656 - acc: 0.5361 - val_loss: 1.0968 - val_acc: 0.6141
Epoch 4/32
500/500 [==============================] - 65s 129ms/step - loss: 1.0322 - acc: 0.6360 - val_loss: 0.9475 - val_acc: 0.6737
Epoch 5/32
500/500 [==============================] - 64s 129ms/step - loss: 0.9021 - acc: 0.6850 - val_loss: 0.8354 - val_acc: 0.7085
Epoch 6/32
500/500 [==============================] - 65s 129ms/step - loss: 0.7941 - acc: 0.7225 - val_loss: 0.7644 - val_acc: 0.7437
Epoch 7/32
500/500 [==============================] - 63s 126ms/step - loss: 0.7357 - acc: 0.7